# modframe — interactive example & workspace index

This notebook provides smoke tests, executable examples, scaffolding utilities (README/`outline.md` templates) for the `topics/` hierarchy, and CI/docs integration examples.

Run cells sequentially; deterministic asserts are included so the notebook can be tested with `nbval`.

---

**Purpose:** make it trivial to preview, generate, and validate workspace artifacts and interactive examples.

In [ ]:
import sys, subprocess, textwrap

print("Repo smoke test — attempt to import `modframe`")
cmd = [sys.executable, "-c", "import modframe, inspect; print(getattr(modframe, '__version__', 'no-version')); print([n for n in dir(modframe) if not n.startswith('_')])"]
try:
    out = subprocess.check_output(cmd, stderr=subprocess.STDOUT, text=True, timeout=10)
    print(out)
    smoke_ok = True
except Exception as e:
    print("modframe import failed:", e)
    smoke_ok = False

print("smoke_ok =", smoke_ok)
# For CI, enable the next line to fail the job if import fails:
# assert smoke_ok

## Create notebook file — programmatic demo

The cell below demonstrates how to create a small notebook programmatically (writes `modframe/interactive_example_generated.ipynb`). This is useful for reproducible examples or generating classroom materials.

In [ ]:
from pathlib import Path
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell, new_code_cell

nb = new_notebook(cells=[
    new_markdown_cell("# Auto-generated notebook\nThis file is created programmatically as a demo."),
    new_code_cell("print('hello from generated notebook')"),
])
out_path = Path('modframe/interactive_example_generated.ipynb')
nbformat.write(nb, out_path.open('w', encoding='utf-8'))
print('Wrote', out_path)

## API walkthrough — core classes & functions

Attempt to import `modframe` and introspect core public symbols (safe to run even if package is not installed).

In [ ]:
import importlib, inspect

try:
    pkg = importlib.import_module('modframe')
    public = [n for n in dir(pkg) if not n.startswith('_')]
    print('modframe imported — public symbols:', public)
    for name in public[:6]:
        obj = getattr(pkg, name)
        if callable(obj):
            try:
                print(f'\n{name} signature:', inspect.signature(obj))
            except Exception:
                print(f'\n{name} — no signature available')
except Exception as e:
    print('modframe not importable:', e)
    print('Tip: run `pip install -e .` or add package to PYTHONPATH')

## Usage examples — end-to-end workflow (deterministic)

This cell demonstrates a small, deterministic data pipeline so `nbval` can validate the outputs.

In [ ]:
import pandas as pd

df = pd.DataFrame({'id':[1,2,3],'value':[10,20,30]})
df['value2'] = df['value'] * 2
assert df['value2'].tolist() == [20,40,60]
df

## Interactive visualization & widgets (matplotlib / ipywidgets)

The cell below draws a deterministic plot; optional `ipywidgets` code is commented so the notebook stays runnable in minimal envs.

In [ ]:
import matplotlib.pyplot as plt

def sample_plot(scale=1.0):
    xs = [0,1,2,3,4]
    ys = [scale * (x+1) for x in xs]
    plt.figure(figsize=(5,2.5))
    plt.plot(xs, ys, marker='o')
    plt.title(f'Sample plot — scale={scale}')
    plt.show()

sample_plot(1.0)

# Uncomment the following lines if `ipywidgets` is installed:
# from ipywidgets import interact
# interact(sample_plot, scale=(0.5, 3.0, 0.5))

## Make notebook testable with `nbval`

- Mark deterministic cells with `assert` so `nbval` can validate outputs.
- Example test command: `pytest --nbval modframe/interactive_example.ipynb`

In [ ]:
print('CI/test command example: pip install -e .[test] nbval && pytest --nbval modframe/interactive_example.ipynb')

## Convert notebook to docs (nbconvert / nbsphinx / mkdocs-jupyter)

Suggested commands:

```bash
jupyter nbconvert --to rst modframe/interactive_example.ipynb
# or configure nbsphinx/mkdocs-jupyter in your docs pipeline
```


## CI: run notebook tests in GitHub Actions

Example job step (add to your test workflow):

```yaml
- name: Test notebooks
  run: |
    pip install -e .[test] nbval
    pytest --nbval modframe/interactive_example.ipynb
```


## Rename folder & update imports — suggested alternate names

Suggested alternate names (pick one):
- `modframe_core`  (explicit, package-like)
- `framekit`       (short & brandable)
- `policyframe`    (descriptive)
- `systemlens`     (audience-friendly)
- `struct-activation` (project-specific)

Safe rename recipe (run manually when ready):

```bash
git mv modframe modframe_core
git grep -l "\bmodframe\b" | xargs sed -i 's/modframe/modframe_core/g'
git add -A && git commit -m "rename modframe -> modframe_core"
pytest  # run tests
```


## Post-rename: run tests, lint, and update packaging

After rename run:

```bash
pytest
ruff .
mypy .
# update `pyproject.toml` / packaging name if needed
python -m build
```

This ensures imports, CI, and packaging remain healthy after a refactor.